* Firstly, let's add WaveBeans dependencies as well as logging library.
* Secondly, add all imports that might be useful.

In [1]:
%use lets-plot

@file:Repository("file:///home/jovyan/maven-local/repository")
@file:Repository("https://dl.bintray.com/wavebeans/wavebeans")
@file:DependsOn("io.wavebeans:lib:0.0.3-SNAPSHOT4")
@file:DependsOn("io.wavebeans:exe:0.0.3-SNAPSHOT4")
@file:DependsOn("io.wavebeans:http:0.0.3-SNAPSHOT4")
@file:DependsOn("ch.qos.logback:logback-classic:1.2.3")
@file:DependsOn("ch.qos.logback:logback-core:1.2.3")

import io.wavebeans.lib.*
import io.wavebeans.lib.io.*
import io.wavebeans.lib.math.*
import io.wavebeans.lib.stream.*
import io.wavebeans.lib.stream.fft.*
import io.wavebeans.lib.stream.window.*
import io.wavebeans.lib.table.*
import io.wavebeans.http.*
import io.wavebeans.execution.*
import java.util.concurrent.TimeUnit.*

* Start the HTTP Service on port 6800, so we could query all data in a different Jupyter notebook

In [2]:
HttpService(serverPort = 6800).start()

io.wavebeans.http.HttpService@36e2c864

* Define our signal processing stream:
    * it is infinite;
    * we store only last 10 seconds in the table.

In [3]:
val o = (880.sine() + 440.sine() + 220.sine())
    .window(401, step = 128)
    .fft(512)
    .toTable("fft", 2.m)

* Execute stream, it'll never finish though

In [ ]:
val overseer = LocalOverseer(listOf(o))
overseer.eval(44100.0f).all { it.get().finished }
overseer.close()